# Run functions from XHARPy from QCrBox

XHARPy is an external library to calculate atomic from factors from periodic PAW DFT calculations availablable on its own on [Github](https://github.com/Niolon/XHARPy). Two functionalities are currently exposed within QCrBox: calculation of atomic form factors and full HAR refinement.

The example starts again by using the dotenv package. It makes things more convenient as we can read the environment variables from the .env.dev file in the QCrBox directory. If you want to run with python core packages only, comment out the load_dotenv lines and insert the local and qcrbox pathes manually into the `pathhelper` using the information in your .env.dev file.

In [1]:
import os
import shutil

from dotenv import load_dotenv

from qcrbox_wrapper import QCrBoxWrapper, QCrBoxPathHelper


if not load_dotenv('../.env.dev'):
    raise FileNotFoundError(
        ".dot.env file could not be loaded. Either adapt the path to your filesystem or "
        + "input the information loaded from os.environ manually"
    )


We create an example folder to try out this functionality.

In [2]:
pathhelper = QCrBoxPathHelper(
    os.environ['QCRBOX_SHARED_FILES_DIR_HOST_PATH'],
    os.environ['QCRBOX_SHARED_FILES_DIR_CONTAINER_PATH'],
    'examples_xharpy'
)

path_local = pathhelper.local_path
path_qcrbox = pathhelper.qcrbox_path

## Connecting to QCrBox
We can connect to the QCrBox Inventory via python after we have started everything with qcb up. Sometimes the server takes a while so you might need to retry if it initially refuses connection. This should not take more than 30 seconds after your console output says that everything has started.

In [3]:
qcrbox = QCrBoxWrapper('127.0.0.1', 11000)

In [4]:
qcrbox.application_dict

{'XHARPy-GPAW': XHARPy-GPAW()}

In [5]:
xharpy = qcrbox.application_dict['XHARPy-GPAW']

In [6]:
help(xharpy)

Help on QCrBoxApplication in module qcrbox_wrapper:

XHARPy-GPAW()
    Represents the XHARPy-GPAW application (v. 0.2.0) in QCrBox
    
    Methods:
        atom_form_fact_gpaw(
            input_cif_path: str,
            output_tsc_path: str,
            functional: str,
            gridspacing: str
        )
    
        ha_refine(
            input_cif_path: str,
            output_cif_path: str,
            functional: str,
            gridspacing: str
        )



## Calculating aspheric atomic form factors and writing them into a tsc file

A `tsc` file is the current standard way of exchanging non-spheric atomic form factors. Here we can calculate the tsc for the atoms within our asymmetric unit for the given crystal structure using GPAW. Functionals are any functionals that GPAW can use. A smaller grid spacing means more grid points *i.e.* a larger basis set in our calculation.


In [7]:
# create a new folder for this example
folder_aff = path_local / 'run_tsc_calc'
folder_aff.mkdir(exist_ok=True)

# copy file
shutil.copy('./input_files/input.cif', folder_aff / 'input.cif')

# start command
calc = xharpy.atom_form_fact_gpaw(
    input_cif_path=path_qcrbox / 'run_tsc_calc' / 'input.cif',
    output_tsc_path=path_qcrbox / 'run_tsc_calc' / 'affs.tsc',
    functional='PBE',
    gridspacing=0.20
)

# wait for command to finish
calc.wait_while_running(1.0)

## Running a Hirshfeld Atom Refinement
XHARPy can also refine a structure directly. As the XHARPy refinement is slower than that of Olex2 this should probably be seen as an example more than a direct implementation. Functionals are any functionals that GPAW can use. A smaller grid spacing means more grid points *i.e.* a larger basis set in our calculation.

In [8]:
# create a new folder for this example
folder_har = path_local / 'run_har'
folder_har.mkdir(exist_ok=True)

# copy file
shutil.copy('./input_files/input.cif', folder_har / 'input.cif')

# start command
calc2 = xharpy.ha_refine(
    input_cif_path=path_qcrbox / 'run_har' / 'input.cif',
    output_cif_path=path_qcrbox / 'run_har' / 'output.cif',
    functional='PBE',
    gridspacing=0.20
)

# wait for command to finish
print(calc2.status)
calc2.wait_while_running(1.0)
print(calc2.status)

{'response_to': 'invoke_command', 'status': 'error', 'msg': None, 'payload': {'msg_execute_calculation': {'action': 'execute_calculation', 'payload': {'command_id': 2, 'calculation_id': 2, 'arguments': {'input_cif_path': '/mnt/qcrbox/shared_files/examples_xharpy/run_har/input.cif', 'output_cif_path': '/mnt/qcrbox/shared_files/examples_xharpy/run_har/output.cif', 'functional': 'PBE', 'gridspacing': '0.2'}, 'container_qcrbox_id': 'qcrbox_container_0x33039282569244f2aca022ca50206ced'}}, 'routing_key': 'qcrbox_rk_0x3120bbd76e0c478991e965a71890316d'}}


ConnectionError: Command not successfully send